In [19]:
!pip install beautifulsoup4
!pip install lxml
!conda install -c conda-forge folium=0.5.0 --yes
!pip install geocoder
!conda install -c conda-forge geopy --yes

distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


Solving environment: ...working... done

# All requested packages already installed.



distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\warmb\Anaconda3

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    geopy-1.19.0               |             py_0          53 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          85 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.49-py_0
  geopy              conda-forge/noarch::geopy-1.19.0-py_0




geographiclib-1.49   | 32 KB     |            |   0% 
geographiclib-1.49   | 32 KB     | ###7       |  37% 
geographiclib-1.49   | 32 KB     | ########## | 100% 

geopy-1.19.0         | 53 KB     |            |   0% 
geopy-1.19.0         | 53 K

In [21]:
import pandas as pd 
import numpy as np
import folium
from bs4 import BeautifulSoup
import requests
from geopy.geocoders import Nominatim

In [2]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')

In [3]:
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );
  </script>
  <script>
   (window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":890001695,"wgRevisionId":890001695,"wgArticleId":539066,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Communications in Ontario","Postal codes in Canada","Toronto","Ontario-related lists"],"wgBreakFrames":false,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wg

In [4]:
table = soup.find('table').tbody
column_names = []
for item in table.find_all('th'):
    column_names.append(item.text.strip())
neighborhoods = pd.DataFrame(columns=column_names)
neighborhoods

,Postcode,Borough,Neighbourhood


In [5]:
neighborhoods = pd.DataFrame(columns=column_names)
for item in table.find_all('tr'):
    assigned = True
    temp = item.find_all('td')
    if len(temp) == 0: 
        continue
    postcode = temp[0].text.strip()
    borough = temp[1].text.strip()
    neighbor = temp[2].text.strip()
    if (borough == 'Not assigned'):
        continue
    elif (neighbor == 'Not assigned'):
        neighbor = borough
    neighborhoods = neighborhoods.append({column_names[0]: postcode,
                                          column_names[1]: borough,
                                          column_names[2]: neighbor}, ignore_index=True)


In [6]:
neighborhoods.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [7]:
def f(x):
     return pd.Series(dict(Borough = x['Borough'].unique()[0], Neighbourhood = "%s" % ', '.join(x['Neighbourhood'])))
group_neighborhoods = neighborhoods.groupby('Postcode').apply(f).reset_index()

In [8]:
group_neighborhoods.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [9]:
print(group_neighborhoods.shape)

(103, 3)


In [10]:
data_loc = pd.read_csv("Geospatial_Coordinates.csv") 
data_loc.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [14]:
result = pd.merge(group_neighborhoods, data_loc, left_on='Postcode', right_on='Postal Code')
result.drop(['Postal Code'], axis=1,inplace=True)
result.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [17]:
toronto_data = result[result['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [22]:
# create map of Manhattan using latitude and longitude values
address = 'Toronto, CA'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [24]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

The map shows that the clusters of locaitons spread across Toronto with a cluster in the north area. 